In [2]:
"""
extra_validation.py

Optional, exploratory validation script for the ECLSS synthetic dataset.

This script is meant to be run AFTER the dataset is generated by the main
data generation script. It loads:

  - cycles_3d.npy
  - labels_system.npy

from the output directory (default: "eclss_synthetic_dataset_full")
and performs additional, *non-mandatory* checks:

  1) Per-class summary statistics (mean / std per sensor)
  2) KS tests on per-cycle statistics within the nominal class
  3) Cross-correlations between sensors in the nominal class
  4) Approximate signal-to-noise ratios (SNR) for the nominal class

IMPORTANT:
  - This script is exploratory only. It does NOT enforce pass/fail thresholds.
  - Some tests (e.g., KS and SNR) are very sensitive and are interpreted
    qualitatively rather than as hard validity checks.
"""

"""
extra_validation.py
Exploratory validation script for ECLSS dataset.
"""

from __future__ import annotations
import numpy as np
import os
from pathlib import Path

# ---------------------------------------------------------
# FIXED: Windows path to dataset output directory
# ---------------------------------------------------------
DATA_DIR = r"C:\Users\ahasa\project_root\data\eclss_synthetic_dataset_full"

CYCLES_FILE = os.path.join(DATA_DIR, "cycles_3d.npy")
LABELS_FILE = os.path.join(DATA_DIR, "labels_system.npy")


def load_data():
    print("=" * 72)
    print(" LOADING DATA ")
    print("=" * 72)

    data_3d = np.load(CYCLES_FILE)       # shape (N, T, 3)
    labels_system = np.load(LABELS_FILE) # shape (N,)

    print(f"Loaded cycles from: {CYCLES_FILE}")
    print(f"Loaded labels from: {LABELS_FILE}")
    print(f"Data shape: {data_3d.shape} (N, T, C)")
    print(f"Unique system classes: {np.unique(labels_system)}")
    print()

    return data_3d, labels_system


def per_class_summary(data_3d: np.ndarray, labels_system: np.ndarray) -> None:
    """
    Print mean / std per sensor for each system class.
    """
    print("=" * 72)
    print(" 1) PER-CLASS SUMMARY STATISTICS ")
    print("=" * 72)

    sensors = ["O2", "CO2", "Pressure"]
    n_classes = int(labels_system.max()) + 1

    for c in range(n_classes):
        idx = np.where(labels_system == c)[0]
        class_data = data_3d[idx]  # (Nc, T, 3)
        if class_data.size == 0:
            continue

        print(f"\nClass {c}: N={len(idx)}")
        for s_idx, s_name in enumerate(sensors):
            x = class_data[:, :, s_idx].ravel()
            print(f"  {s_name}: mean={x.mean():.4f}, std={x.std():.4f}")
    print()


def ks_on_per_cycle_stats(data_3d: np.ndarray, labels_system: np.ndarray) -> None:
    """
    Perform KS tests on per-cycle statistics (mean values) within the nominal class.

    Instead of flattening entire time-series (which yields huge sample sizes and
    oversensitive p-values), we compute a single scalar per cycle (mean over time)
    and compare two random halves of the nominal set.

    This is a much more reasonable use of KS for this context.
    """
    from scipy.stats import ks_2samp

    print("=" * 72)
    print(" 2) KS TEST ON PER-CYCLE STATISTICS (NOMINAL CLASS) ")
    print("=" * 72)

    sensors = ["O2", "CO2", "Pressure"]

    # Nominal class is assumed to be class 0
    nominal_idx = np.where(labels_system == 0)[0]
    nominal_data = data_3d[nominal_idx]  # (N_nominal, T, 3)

    N_nominal = nominal_data.shape[0]
    if N_nominal < 4:
        print("Not enough nominal samples for a meaningful KS split (need >= 4).")
        print()
        return

    # Shuffle indices for a fair split
    rng = np.random.default_rng(123)
    perm = rng.permutation(N_nominal)
    half = N_nominal // 2

    group1_idx = perm[:half]
    group2_idx = perm[half:]

    group1 = nominal_data[group1_idx]
    group2 = nominal_data[group2_idx]

    # Compute per-cycle mean per sensor
    g1_means = group1.mean(axis=1)  # (half, 3)
    g2_means = group2.mean(axis=1)  # (N_nominal - half, 3)

    print(f"Nominal samples: {N_nominal}")
    print(f"Group sizes: {group1.shape[0]} vs {group2.shape[0]} (per-cycle means)\n")
    print("KS tests on per-cycle mean values (p-values only, no hard thresholds):\n")

    for s_idx, s_name in enumerate(sensors):
        stat, p = ks_2samp(g1_means[:, s_idx], g2_means[:, s_idx])
        print(f"  {s_name}: KS statistic={stat:.4f}, p-value={p:.4f}")

    print("\nInterpretation:")
    print("  - Very small p-values indicate noticeable differences between subsets of")
    print("    nominal cycles, which can arise from random drift/noise.")
    print("  - Since drift and jitter are intentionally included, some differences")
    print("    are expected and not necessarily problematic.")
    print()


def cross_correlations_nominal(data_3d: np.ndarray, labels_system: np.ndarray) -> None:
    """
    Compute simple cross-correlations between sensors in the nominal class
    using the mean nominal cycle.
    """
    print("=" * 72)
    print(" 3) CROSS-CORRELATIONS (NOMINAL CLASS) ")
    print("=" * 72)

    nominal_idx = np.where(labels_system == 0)[0]
    nominal_data = data_3d[nominal_idx]  # (N_nominal, T, 3)
    if nominal_data.size == 0:
        print("No nominal samples found (class 0).")
        print()
        return

    mean_cycle = nominal_data.mean(axis=0)  # (T, 3)
    corr = np.corrcoef(mean_cycle.T)       # (3, 3)

    print("Correlation matrix between (O2, CO2, Pressure) using mean nominal cycle:")
    print(corr)
    print()
    print(f"O2–CO2 correlation:  {corr[0, 1]:+0.3f}")
    print(f"O2–Pressure:         {corr[0, 2]:+0.3f}")
    print(f"CO2–Pressure:        {corr[1, 2]:+0.3f}")
    print()
    print("Interpretation:")
    print("  - Strong negative O2–CO2 correlation (near -1) is expected due to")
    print("    inverse sinusoidal behavior (adsorption/desorption).")
    print("  - O2–Pressure and CO2–Pressure close to 0 indicate decoupled pressure")
    print("    dynamics, which matches the modeling assumptions.")
    print()


def snr_nominal(data_3d: np.ndarray, labels_system: np.ndarray) -> None:
    """
    Compute a rough signal-to-noise ratio (SNR) for nominal cycles per sensor.

    Definition here:
      - mean_cycle(t) = average over nominal samples
      - signal_power   = var(mean_cycle(t))
      - noise_power    = var(x - mean_cycle) over all nominal samples
      - SNR_dB         = 10 * log10(signal_power / noise_power)

    This is a heuristic measure and is reported without strict thresholds.
    """
    print("=" * 72)
    print(" 4) APPROXIMATE SNR (NOMINAL CLASS) ")
    print("=" * 72)

    nominal_idx = np.where(labels_system == 0)[0]
    nominal_data = data_3d[nominal_idx]  # (N_nominal, T, 3)
    if nominal_data.size == 0:
        print("No nominal samples found (class 0).")
        print()
        return

    sensors = ["O2", "CO2", "Pressure"]

    for s_idx, s_name in enumerate(sensors):
        sensor_data = nominal_data[:, :, s_idx]  # (N_nominal, T)
        mean_signal = sensor_data.mean(axis=0)   # (T,)

        signal_power = np.var(mean_signal)
        noise = sensor_data - mean_signal[np.newaxis, :]
        noise_power = np.var(noise)

        if noise_power <= 0.0:
            snr_db = float("inf")
        else:
            snr_db = 10.0 * np.log10(signal_power / noise_power)

        print(f"{s_name}: SNR ≈ {snr_db:5.2f} dB")

    print()
    print("Interpretation:")
    print("  - These SNR values reflect the chosen amplitude, drift, and noise levels.")
    print("  - Lower SNR implies more challenging detection, which can be desirable")
    print("    for robust anomaly detection benchmarks.")
    print("  - There is no strict 'required' SNR here; these numbers are reported")
    print("    for transparency and discussion rather than as hard constraints.")
    print()


def main():
    data_3d, labels_system = load_data()

    # 1) Per-class means/stds
    per_class_summary(data_3d, labels_system)

    # 2) KS test on per-cycle statistics (nominal class)
    try:
        ks_on_per_cycle_stats(data_3d, labels_system)
    except ImportError:
        print("scipy not available; skipping KS tests.\n")

    # 3) Cross-correlations in nominal class
    cross_correlations_nominal(data_3d, labels_system)

    # 4) Approximate SNR in nominal class
    snr_nominal(data_3d, labels_system)

    print("=" * 72)
    print(" EXTRA VALIDATION COMPLETE (EXPLORATORY) ")
    print("=" * 72)
  

if __name__ == "__main__":
    main()


 LOADING DATA 
Loaded cycles from: C:\Users\ahasa\project_root\data\eclss_synthetic_dataset_full\cycles_3d.npy
Loaded labels from: C:\Users\ahasa\project_root\data\eclss_synthetic_dataset_full\labels_system.npy
Data shape: (720, 1000, 3) (N, T, C)
Unique system classes: [0 1 2 3 4 5]

 1) PER-CLASS SUMMARY STATISTICS 

Class 0: N=120
  O2: mean=20.9029, std=0.2277
  CO2: mean=0.3012, std=0.0826
  Pressure: mean=14.7015, std=0.2218

Class 1: N=120
  O2: mean=20.9021, std=0.2190
  CO2: mean=0.3561, std=0.1432
  Pressure: mean=14.6978, std=0.2213

Class 2: N=120
  O2: mean=20.9029, std=0.2256
  CO2: mean=0.2981, std=0.0799
  Pressure: mean=14.6995, std=0.2187

Class 3: N=120
  O2: mean=20.9021, std=0.2227
  CO2: mean=0.2981, std=0.0816
  Pressure: mean=14.6355, std=0.3767

Class 4: N=120
  O2: mean=20.9027, std=0.2237
  CO2: mean=0.5709, std=0.1628
  Pressure: mean=14.7001, std=0.2222

Class 5: N=120
  O2: mean=18.6533, std=1.0466
  CO2: mean=0.2980, std=0.0787
  Pressure: mean=14.6992, s